# Construct an *E. coli* Cell Representation

In this tutorial, you will learn how to construct a cell and add features at specific locations. We will construct an model of an *E. coli* cell using the Builder feature and idealized shapes provided by Lattice Microbes. Once built, ribosomes will be placed at specific starting locations that were extracted from Cryo-electron microscopy (Cryo-EM) data. Finally, a Jupyter GUI will be used to examine the initial conditions.

The specification of an RDME simulation will generally occur in the order:

1. Define simulation volume
2. Define region types
3. Specify reaction model
4. Specify diffusion model
5. Create simulation geometry
6. Customize simulation initial conditions

In [ ]:
import os
import lm
import pyLM
from pyLM.RDME import RDMESimulation
from pyLM.units import *
import pySTDLM
from pyLM.ipyInterface import visualizeRDMEInitialConditions

import numpy as np
import math
import scipy as sp
import scipy.spatial

import matplotlib.pyplot as plt
%matplotlib notebook

## Create an RDME Simulation about the volume of an *E. coli* Cell

First, a simulation domain is created that can hold an *E. coli* cell. From visual inspection of the Cryo-EM data, the cell is about 0.8 $\mu m$ wide and about 3.6 $\mu m$ long. Generally, the long dimension is aligned along the Z-axis, as this is the axis in which the <a href="http://www.sciencedirect.com/science/article/pii/S0167819114000398">domain decomposition occurs allowing parallelization over many GPUs</a>. In this case, we won't actually simulate the system, so it isn't an issue, but in practice it is good.

In [ ]:
sim = RDMESimulation(dimensions=micron(1.024,1.024,4.096), 
                     spacing=nm(16), 
                     name="E. coli with Custom Data", 
                     defaultRegion="extracellular")

## Define the Reaction and Diffusion Models


### Create Region Types

After defining the simulation volume, the types of possible regions are defined. We use "region types" and "site types" interchangabely when talking about simulations. Region types do not define an actual spatial location, but a type of locations that have distinct reaction and diffusion characteristics.

In this simulation, we will have three main region types:

1. membrane - the cell membrane
2. cytoplasm - the region that is generally free of DNA
3. nucleoid - the region with slower diffusion due to ample DNA

In [ ]:
mem = sim.addRegion("membrane")
cyt = sim.addRegion("cytoplasm")
nuc = sim.addRegion("nucleoid")
ext = sim.modifyRegion("extracellular")

### Reaction Model

We define a toy gene expression model (with reasonable rates):

1. D - Gene
2. M - mRNA transcribed from gene
3. P - protein translated from mRNA
4. R - ribosomes

Production reactions

$D \longrightarrow D+M$

$M \longrightarrow M+P$

$R+M \longrightarrow R+M+P$

Degradation reactions

$M \longrightarrow \emptyset$

$P \longrightarrow \emptyset$

In [ ]:
# Define the species
species = ['D','M','P','R']
sim.defineSpecies(species)

# Production Reactions
nuc.addReaction(reactant='D', product=('D','M'), rate=0.1)
for reg in [nuc, cyt, mem]:
    reg.addReaction(reactant=('R','M'), product=('R','M','P'), rate=1e-6)

# Degradation reactions
for reg in [nuc, cyt, mem]:
    reg.addReaction(reactant='M', product='', rate=0.0033)
    reg.addReaction(reactant='P', product='', rate=1.0/(20.0*60.0))

### Diffusion Model

Next, we define how particles diffuse in and among different region types. Diffusion rates are in MKS units (e.g. $m^2/s$)

Two types of diffusions need to be defined:

1. Intra-region diffusion - set with code of the form `region.setDiffusionRate(species=..., rate=...)`
2. Inter-region diffusion - set with code ofthe form `simulation.setTwoWayTransitionRate(species=..., one=..., two=..., rate=...)`

In [ ]:
# mRNA diffusion dates
nuc.setDiffusionRate(species='M', rate=5e-14)
cyt.setDiffusionRate(species='M', rate=5e-13)
sim.setTwoWayTransitionRate(species='M',one='nucleoid', two='cytoplasm', rate=5e-13)
sim.setTwoWayTransitionRate(species='M',one='membrane', two='cytoplasm', rate=5e-13)

# Protein diffusion rates
nuc.setDiffusionRate(species='P', rate=5e-13)
cyt.setDiffusionRate(species='P', rate=5e-12)
sim.setTwoWayTransitionRate(species='P',one='nucleoid', two='cytoplasm', rate=5e-12)
_ = sim.setTwoWayTransitionRate(species='P',one='membrane', two='cytoplasm', rate=5e-12)


### Add Particles that are Randomly Distributed

When the starting location of particles isn't important, we just call the `sim.addParticles(...)` function. At the time of discretization (see below) particles will be randomly placed within the volumes that are defined as each region type.

In [ ]:
sim.addParticles(species='D', region='nucleoid', count=1)
sim.addParticles(species='M', region='cytoplasm', count=30)
sim.addParticles(species='P', region='cytoplasm', count=1000)

# Display the simulation setup
sim

## Build Spatial Regions

Next we will define the spatial model for the simulation, in other words, we will define which parts of the simulation domain correspond to which region types. We will create a cell that is 3.6 $\mu m$ long with a radius of 0.4 $\mu m$. Additionally, the membrane on the outside will be 32 $nm$ wide. This is unrealistically large, however, it will ensure that there are no holes in the membrane.

In [ ]:
radius = micron(0.4) # Radius of the cell
length = micron(3.6) # Length of a cell 
membraneThickness = nm(32) # Membrane thickness (2 lattice sites to prevent holes)

### Cell Membrane and Cytoplasm

The cell builder code in Lattice Microbes works like a paint canvas. Because each location in the lattice can only correspond with a single region type, the most recently applied shape will overwrite any previous applied. In this way, we will "paint" the structures one; first a spherocylinder for the membranem type, then another spherocylinder slightly smaller with a cytoplasm type, finally a much smaller spherocylinder for the nucleoid type.

In [ ]:
# Points take absolue coordinates
p1 = lm.point(micron(0.512),micron(0.512), micron(4.096)/2.0-(length/2.0 - radius))
p2 = lm.point(micron(0.512),micron(0.512), micron(4.096)/2.0+(length/2.0 - radius))

# Construct a capsule for the outside of the membrane
cellMembrane = lm.Capsule(p1, p2, radius, sim.siteTypes['membrane'])

# Construct a capsule for the outside of the membrane
cellCytoplasm = lm.Capsule(p1, p2, radius-membraneThickness, sim.siteTypes['cytoplasm'])

### Cell Nucleoid

In [ ]:
# Construct a capsule for the outside of the membrane
p3 = lm.point(micron(0.512),micron(0.512), micron(4.096)/2.0-(0.8*length/2.0 - 0.7*radius))
p4 = lm.point(micron(0.512),micron(0.512), micron(4.096)/2.0+(0.8*length/2.0 - 0.7*radius))
cellNucleoid = lm.Capsule(p3, p4, 0.7*radius, sim.siteTypes['nucleoid'])

### Add shapes to the LM Builder

In the last two code cells, we created three objects: cellMembrane, cellCytoplasm and cellNucleoid. Once create, we can layer these objects by adding them to the lm_builder object that is associated with the RDMESimulation object. This is the act of "painting" them.

In [ ]:
# Layer each object into the simulation domain (this paints each layer in order)
sim.lm_builder.addRegion(cellMembrane)
sim.lm_builder.addRegion(cellCytoplasm)
sim.lm_builder.addRegion(cellNucleoid)

## Customize Spatial Regions

Once the reaction, diffusion and spatial models have been defined, we can customize the simulation volume by adding particles at specific locations or modifying individual or sets of specific lattice sites. This is generally the location in simulation setup where any spatial data from experiments can be integrated. 

In this example, we will add ribosomes that were extracted from Cryo-EM data. The data is for a half of a single *E. coli* cell in exponential phase growing in glucose supplemented with amino acids (for more information <a href="http://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1002010">see this article</a>; esp. Figure 14).

### Load and Transform Ribosome Data

The Cryo-EM data was pre-processed and is available in XYZ coordinates. Because the data is only for half a cell, we will mirror it. Also, because the data was for a cell that was randomly oriented, we will apply translation, rotation and scale operations to fit within our cell. While this operation could be done in a programmatic, intelligent fashion, for brevity we hardcoded them here.

In [ ]:
# Load Ribosome data
ribosomeLocations = np.loadtxt("MOTL_70S_bin2_2_coordinatesINnm_1364p_t14.txt")

# Mirror data because we only have 1/2 of cell
allRibosomes = np.concatenate((ribosomeLocations, -ribosomeLocations + [800,2550,450]))

# Apply transformation to go from data units to units relevant for the cell simulation
# Center on 0
allRibosomes -= np.mean(allRibosomes,0)

# Apply a rotation to align cell along z axis
def eulerAnglesToRotationMatrix(theta) :
    R_x = np.array([[1,         0,                  0                   ],
                    [0,         math.cos(theta[0]), -math.sin(theta[0]) ],
                    [0,         math.sin(theta[0]), math.cos(theta[0])  ]
                    ])           
    R_y = np.array([[math.cos(theta[1]),    0,      math.sin(theta[1])  ],
                    [0,                     1,      0                   ],
                    [-math.sin(theta[1]),   0,      math.cos(theta[1])  ]
                    ])
    R_z = np.array([[math.cos(theta[2]),    -math.sin(theta[2]),    0],
                    [math.sin(theta[2]),    math.cos(theta[2]),     0],
                    [0,                     0,                      1]
                    ])          
    R = np.dot(R_z, np.dot( R_y, R_x ))
    return R

allRibosomes = np.dot(allRibosomes,eulerAnglesToRotationMatrix(np.array([89.0,0.0,-9.0])*np.pi/180.0))

# Center a second time
allRibosomes -= np.mean(allRibosomes,0)

# Scale data to fit within our cell
allRibosomes *= micron(0.7,0.7,3.35)/(np.max(allRibosomes,0)-np.min(allRibosomes,0))


In [ ]:
# Plot the ribosome data to ensure it looks good
fig, axs = plt.subplots(1,3,figsize=(12,3))
axs[0].scatter(allRibosomes[:,0],allRibosomes[:,1])
axs[0].set_xlabel("x")
axs[0].set_ylabel("y")
axs[0].set_xlim(-5e-7,5e-7)
axs[0].set_ylim(-5e-7,5e-7)
c1 = plt.Circle((0,0), 4e-7, color='r', alpha=0.3)
axs[0].add_artist(c1)

axs[1].scatter(allRibosomes[:,0],allRibosomes[:,2])
axs[1].set_xlabel("x")
axs[1].set_ylabel("z")
axs[1].set_xlim(-10e-7,10e-7)
axs[1].set_ylim(-2e-6,2e-6)
c1 = plt.Circle((0,1.4e-6), 4e-7, color='r', alpha=0.3)
c2 = plt.Circle((0,-1.4e-6), 4e-7, color='r', alpha=0.3)
axs[1].add_artist(c1)
axs[1].add_artist(c2)

axs[2].scatter(allRibosomes[:,1],allRibosomes[:,2])
axs[2].set_xlabel("y")
axs[2].set_ylabel("z")
axs[2].set_xlim(-10e-7,10e-7)
axs[2].set_ylim(-2e-6,2e-6)
c1 = plt.Circle((0,1.4e-6), 4e-7, color='r', alpha=0.3)
c2 = plt.Circle((0,-1.4e-6), 4e-7, color='r', alpha=0.3)
axs[2].add_artist(c1)
axs[2].add_artist(c2)


plt.tight_layout()
plt.show()

### Add Ribosomes at Correct Locations

In [ ]:
# Add Ribosome Particles to the simulation
l = sim.getLattice()
for i,j in enumerate(allRibosomes):
    idx = list(map(int,(j+micron(0.5,0.5,2.0))/sim.latticeSpacing))
    sim.addParticleAtIdx(index=idx,particleType='R')

## Visualize Simulation Intial Conditions

Finally, we can visualize the cell. To do this, we will use a function in pyLM that allows interactive visualization inside the Jupyter notebook. The GUI allows you to navigate through slices along the different axes (x, y, or z) and display different sets of particles (by shift- or control-clicking). Additionally, you can click within the image to get information about the site type and the particles contained within.

In [ ]:
# Call this function to create an interactive widget in the Jupyter notebook to visualize data.
visualizeRDMEInitialConditions(sim)

In [ ]:
# Save the simulation file to allow visualization with external software (e.g. VisIt or VMD)
try:
    os.remove('tut2.4-Ecolicell.lm')
except:
    pass
sim.save("tut2.4-Ecolicell.lm")